In [33]:
import pandas as pd
import docx
import re

In [ ]:
doc_path = 'MS.docx' 
doc = docx.Document(doc_path)

def get_first_two_unique_columns(df):
    # Ensure there are at least two columns
    if df.shape[1] < 2:
        raise ValueError("DataFrame must have at least two columns")
    
    first_col = df.iloc[:, 0]  # Always take the first column
    selected_columns = [first_col]
    
    # Iterate through remaining columns to find a unique second column
    for i in range(1, df.shape[1]):
        candidate_col = df.iloc[:, i]
        
        # Ignore the first 3 rows when checking for uniqueness
        if not first_col.iloc[3:].equals(candidate_col.iloc[3:]):
            selected_columns.append(candidate_col)
            break
    
    # If we couldn't find a second unique column, raise an error
    if len(selected_columns) < 2:
        raise ValueError("Could not find two unique columns in the DataFrame")
    
    # Create new DataFrame with selected columns
    return pd.DataFrame({"Col1": selected_columns[0], "Col2": selected_columns[1]})

all_tables = [] 

for table in doc.tables:
    data = []
    for row in table.rows:
        data.append([cell.text.strip() for cell in row.cells])  # Extract text from each cell

    df = pd.DataFrame(data)  # Convert to DataFrame
    df.columns = df.iloc[0]
    df = df[1:]

    df = get_first_two_unique_columns(df)

    df.columns = ['PREFERRED AGENTS', 'NON-PREFERRED AGENTS']


    # same_value_rows = df[df.apply(lambda x: x.nunique() == 1, axis=1)]
    # df_filtered = df.drop(same_value_rows.index).reset_index(drop=True)

    all_tables.append(df)

final_df = pd.concat(all_tables).reset_index(drop=True)
final_df

In [23]:
import pandas as pd
from docx import Document
from docx.shared import Pt

def get_font_size(cell):
    """Return the first explicit run font size (in points) found in this cell,
    or fall back to the paragraph’s style font size."""
    for para in cell.paragraphs:
        for run in para.runs:
            if run.font.size:
                return run.font.size.pt
        if para.style and para.style.font.size:
            return para.style.font.size.pt
    return None

def find_two_unique_cols(table):
    """
    Look at the text of each cell in the table to figure out which two
    columns you actually want:
      - col0 is always the “first” column
      - scan col1, col2… and pick the first one whose text (ignoring
        the first three data rows) isn’t identical to col0
    Returns (first_idx, second_idx).
    """
    # build a text‐only matrix
    data = [[cell.text.strip() for cell in row.cells] for row in table.rows]
    df = pd.DataFrame(data)
    # first row is header
    df.columns = df.iloc[0]
    df = df[1:]
    
    first = df.iloc[:, 0]
    for i in range(1, df.shape[1]):
        cand = df.iloc[:, i]
        # ignore first 3 data‐rows when checking equality
        if not first.iloc[3:].equals(cand.iloc[3:]):
            return 0, i
    
    raise ValueError("Could not find a second unique column in this table")

def extract_pdl_flat(doc_path):
    doc = Document(doc_path)
    records = []
    
    for table in doc.tables:
        # reset headers for each table
        current_main = None
        current_sub  = None
        
        # figure out which two columns hold your PDL drugs
        first_idx, second_idx = find_two_unique_cols(table)
        
        # now walk each data‐row (skip the table’s very first row)
        for row in table.rows[1:]:
            for col_idx, status in ((first_idx, "Preferred"), (second_idx, "Non-Preferred")):
                cell = row.cells[col_idx]
                txt  = cell.text.strip()
                if not txt:
                    continue
                
                size = get_font_size(cell)
                
                # 12 pt = main header
                if size == None:
                    current_main = txt
                    current_sub  = None
                    continue
                
                # 10 pt = subheader
                if size == 10:
                    current_sub = txt
                    continue
                
                #  8 pt = actual drug line
                if size == 8:
                    # stitch together main + sub
                    if current_main and current_sub:
                        tc = f"{current_main}: {current_sub}"
                    elif current_main:
                        tc = current_main
                    else:
                        tc = ""
                    
                    records.append({
                        "pdl_name":          txt,
                        "status":            status,
                        "therapeutic_class": tc
                    })
                    continue
                # else: skip anything else
        
    return pd.DataFrame(records)

# usage
df = extract_pdl_flat("MS.docx")

# df.to_csv("pdl_flat.csv", index=False)


In [ ]:
df['therapeutic_class'] = (
    df['therapeutic_class']
      # remove the asterisked phrase (including the asterisks themselves)
      .str.replace(r'\*[^*]+\*', '', regex=True)
      # collapse any extra spaces that might remain
      .str.replace(r'\s+', ' ', regex=True)
      .str.strip()
)

df['therapeutic_class'] = (
    df['therapeutic_class']
      # remove any "DUR+ " (and trailing spaces)
      .str.replace(r'DUR\+\s*', '', regex=True)
      # collapse extra spaces just in case
      .str.replace(r'\s+', ' ', regex=True)
      .str.strip()
)



,pdl_name,status,therapeutic_class
1939,SOLODYN (minocycline),Non-Preferred,TETRACYCLINES
1940,tetracycline tablet,Non-Preferred,TETRACYCLINES
1941,APRISO (mesalamine),Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
1942,AZULFIDINE (sulfasalazine),Non-Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
1943,balsalazide,Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
1944,COLAZAL (balsalazide),Non-Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
1945,budesonide,Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
1946,DELZICOL (mesalamine),Non-Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
1947,PENTASA (mesalamine),Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
1948,DIPENTUM (olsalazine),Non-Preferred,ULCERATIVE COLITIS & CROHN’S AGENTS : ORAL
